# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
#!pip install --upgrade pip
#!pip install IPython


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib as mpl 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime as dt
import matplotlib.dates as mdates


In [ ]:
values = pd.read_csv('/home/isabela.almeida/Documents/applied-data-representation/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
#print(values.shape)
values = values[~values['Date'].str.contains('02-29')]
values = values[~values['Date'].str.contains('2015')]
#print(values.shape)
values['Data_Value'] = pd.to_numeric(values['Data_Value'])
values['Date'] = pd.to_datetime(values['Date'])
#values = values.drop(columns='ID')

minimum = values[values['Element'] == 'TMIN']
minimum['Data_Value'] = minimum['Data_Value'].div(10)
maximum = values[values['Element'] == 'TMAX']
maximum['Data_Value'] = maximum['Data_Value'].div(10)

minimum['Date'] = minimum['Date'].dt.strftime('%m-%d')
minimum.sort_values(by=['Data_Value'], ascending=True, inplace=True)
minimum.drop_duplicates(subset=['Date'], keep='first', inplace=True)
minimum.sort_values(by=['Date'], ascending=True, inplace=True)

maximum['Date'] = maximum['Date'].dt.strftime('%m-%d')
maximum.sort_values(by=['Data_Value'], ascending=False, inplace=True)
maximum.drop_duplicates(subset=['Date'], keep='first', inplace=True)
maximum.sort_values(by=['Date'], ascending=True, inplace=True)


In [ ]:
scatter = pd.read_csv('/home/isabela.almeida/Documents/applied-data-representation/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
scatter = scatter[~scatter['Date'].str.contains('02-29')]
scatter['Data_Value'] = pd.to_numeric(scatter['Data_Value'])
scatter['Date'] = pd.to_datetime(scatter['Date'])

scatter_min = scatter[scatter['Element'] == 'TMIN']
scatter_min['Data_Value'] = scatter_min['Data_Value'].div(10)
scatter_min['Date'] = scatter_min['Date'].dt.strftime('%m-%d')
scatter_min.sort_values(by=['Data_Value'], ascending=True, inplace=True)
scatter_min.drop_duplicates(subset=['Date'], keep='first', inplace=True)
scatter_min.sort_values(by=['Date'], ascending=True, inplace=True)

scatter_max = scatter[scatter['Element'] == 'TMAX']
scatter_max['Data_Value'] = scatter_max['Data_Value'].div(10)
scatter_max['Date'] = scatter_max['Date'].dt.strftime('%m-%d')
scatter_max.sort_values(by=['Data_Value'], ascending=False, inplace=True)
scatter_max.drop_duplicates(subset=['Date'], keep='first', inplace=True)
scatter_max.sort_values(by=['Date'], ascending=True, inplace=True)


min_points = scatter_min.merge(minimum, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
max_points = scatter_max.merge(maximum, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']


In [ ]:
observation_dates = maximum['Date'].array
plt.figure()
plt.gca().fill_between(maximum['Date'], maximum['Data_Value'], minimum['Data_Value'], facecolor='brown', alpha=0.25)
plt.scatter(min_points['Date'], min_points['Data_Value'], c='#be4588')
plt.scatter(max_points['Date'], max_points['Data_Value'], c='#de6a21')
plt.plot(maximum['Date'], maximum['Data_Value'], '--b', maximum['Date'], minimum['Data_Value'], '-g')

plt.show()
